In [1]:
import os
from random import shuffle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import timedelta
import psycopg2

In [2]:
# config
DataSplatoon = "/home/tidal/ML_Data/Splatoon/input"
Output = "/home/tidal/ML_Data/Splatoon/output"

In [3]:
train = pd.read_csv(DataSplatoon + '/train_data.csv')
test = pd.read_csv(DataSplatoon + '/test_data.csv')

In [4]:
train

,id,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A1-level,...,B2-weapon,B2-rank,B2-level,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level,y
0,1,2019-10-15T20:00:00+00:00,5.0.1,regular,standard,nawabari,sumeshi,sshooter_becchu,NaN,139,...,hokusai_becchu,NaN,26.0,herocharger_replica,NaN,68.0,sharp_neo,NaN,31.0,1
1,2,2019-12-14T04:00:00+00:00,5.0.1,regular,standard,nawabari,arowana,parashelter_sorella,NaN,198,...,squiclean_b,NaN,118.0,campingshelter,NaN,168.0,sputtery_clear,NaN,151.0,0
2,3,2019-12-25T14:00:00+00:00,5.0.1,gachi,standard,hoko,ama,nzap89,a-,114,...,nzap85,a+,163.0,prime_becchu,a-,160.0,dualsweeper_custom,a,126.0,0
3,4,2019-11-11T14:00:00+00:00,5.0.1,regular,standard,nawabari,engawa,bamboo14mk1,NaN,336,...,liter4k,NaN,189.0,promodeler_mg,NaN,194.0,hotblaster_custom,NaN,391.0,0
4,5,2019-12-14T06:00:00+00:00,5.0.1,gachi,standard,hoko,chozame,bold_7,x,299,...,sputtery_hue,x,45.0,bucketslosher_soda,x,246.0,wakaba,x,160.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,66121,2019-10-11T10:00:00+00:00,5.0.1,gachi,standard,area,devon,hokusai_becchu,a,66,...,nzap85,a+,272.0,splatcharger,a,38.0,wakaba,a+,283.0,1
66121,66122,2019-12-04T00:00:00+00:00,5.0.1,gachi,standard,asari,otoro,hokusai_becchu,s+,204,...,prime_becchu,s+,167.0,pablo,s+,188.0,nzap89,s+,170.0,1
66122,66123,2019-10-22T00:00:00+00:00,5.0.1,gachi,standard,hoko,bbass,prime_collabo,a+,116,...,bold_neo,a+,70.0,nzap85,a+,191.0,hissen_hue,a+,139.0,1
66123,66124,2019-12-11T00:00:00+00:00,5.0.1,gachi,standard,area,fujitsubo,heroroller_replica,s+,210,...,heroroller_replica,s+,395.0,rapid_elite_deco,s+,223.0,sharp_neo,s+,194.0,0


In [5]:
test

,id,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A1-level,...,B1-level,B2-weapon,B2-rank,B2-level,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level
0,1,2019-12-17T12:00:00+00:00,5.0.1,gachi,standard,area,hakofugu,prime_collabo,x,174,...,160,nautilus47,x,151.0,l3reelgun_d,x,213.0,nzap89,x,306.0
1,2,2019-11-25T16:00:00+00:00,5.0.1,gachi,standard,asari,anchovy,prime_becchu,s+,363,...,326,dualsweeper_custom,s+,289.0,prime_becchu,s+,147.0,splatroller,s+,156.0
2,3,2019-10-22T08:00:00+00:00,5.0.1,gachi,standard,area,mutsugoro,furo_deco,s,116,...,76,momiji,s,232.0,nzap83,s,183.0,promodeler_pg,s,105.0
3,4,2019-12-30T04:00:00+00:00,5.0.1,gachi,standard,asari,devon,prime_becchu,s+,192,...,90,quadhopper_black,s+,119.0,soytuber,s+,172.0,prime,s+,70.0
4,5,2019-10-15T16:00:00+00:00,5.0.1,gachi,standard,yagura,anchovy,l3reelgun_d,x,267,...,259,sshooter_becchu,x,198.0,splatscope,x,50.0,screwslosher_becchu,x,287.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28335,28336,2019-10-11T16:00:00+00:00,5.0.1,gachi,standard,yagura,battera,prime_becchu,x,67,...,139,hissen,x,152.0,prime_becchu,x,240.0,nzap83,x,150.0
28336,28337,2019-10-20T12:00:00+00:00,5.0.1,gachi,standard,yagura,anchovy,longblaster_necro,c-,25,...,10,bamboo14mk1,c,18.0,nzap89,c-,17.0,dualsweeper,c+,35.0
28337,28338,2019-12-14T00:00:00+00:00,5.0.1,gachi,standard,yagura,zatou,furo,x,386,...,62,bamboo14mk1,x,250.0,carbon_deco,x,257.0,l3reelgun_d,x,243.0
28338,28339,2019-10-12T04:00:00+00:00,5.0.1,gachi,standard,asari,hokke,maneuver_becchu,s+,185,...,89,bold,s+,298.0,hokusai,s+,152.0,ochiba,s+,130.0


In [6]:
train.isnull().sum() #checking for total null values

id                0
period            0
game-ver          0
lobby-mode        0
lobby             0
mode              0
stage             0
A1-weapon         0
A1-rank       14444
A1-level          0
A2-weapon         0
A2-rank       14444
A2-level          0
A3-weapon         0
A3-rank       14444
A3-level          0
A4-weapon        51
A4-rank       14487
A4-level         51
B1-weapon         0
B1-rank       14444
B1-level          0
B2-weapon         0
B2-rank       14444
B2-level          0
B3-weapon         1
B3-rank       14444
B3-level          1
B4-weapon        67
B4-rank       14501
B4-level         67
y                 0
dtype: int64

In [7]:
test.isnull().sum() #checking for total null values

id               0
period           0
game-ver         0
lobby-mode       0
lobby            0
mode             0
stage            0
A1-weapon        0
A1-rank       6272
A1-level         0
A2-weapon        0
A2-rank       6272
A2-level         0
A3-weapon        0
A3-rank       6272
A3-level         0
A4-weapon       15
A4-rank       6280
A4-level        15
B1-weapon        0
B1-rank       6272
B1-level         0
B2-weapon        0
B2-rank       6272
B2-level         0
B3-weapon        1
B3-rank       6273
B3-level         1
B4-weapon       30
B4-rank       6297
B4-level        30
dtype: int64

In [8]:
train[["game-ver"]].apply(pd.value_counts)

,game-ver
5.0.1,66125


In [9]:
train[["period"]].apply(pd.value_counts)

,period
2019-10-12T12:00:00+00:00,249
2019-12-12T14:00:00+00:00,225
2019-10-21T14:00:00+00:00,220
2019-10-23T14:00:00+00:00,216
2019-12-14T16:00:00+00:00,214
...,...
2019-12-10T04:00:00+00:00,3
2019-11-11T02:00:00+00:00,2
2019-12-13T04:00:00+00:00,2
2019-11-15T04:00:00+00:00,1


In [19]:
train['yyyymmdd'] = ""
train["time"] = ""
for i in range(train.shape[0]):
    tmp = train.at[i,"period"].split('T')
    train.at[i,"yyyymmdd"] = tmp[0]
    train.at[i,"time"] = tmp[1]


In [21]:
train[["yyyymmdd"]].apply(pd.value_counts)

,yyyymmdd
2019-10-19,1268
2019-10-12,1232
2019-11-09,1204
2020-01-05,1153
2019-11-03,1131
...,...
2019-11-21,443
2019-11-20,429
2019-11-19,390
2019-11-18,366


In [22]:
train[["time"]].apply(pd.value_counts)

,time
14:00:00+00:00,12016
12:00:00+00:00,10205
16:00:00+00:00,8651
10:00:00+00:00,6565
08:00:00+00:00,4948
06:00:00+00:00,4699
18:00:00+00:00,3933
04:00:00+00:00,3775
02:00:00+00:00,3001
20:00:00+00:00,2993


In [49]:
train["dateDelta"] = ""
for i in range(train.shape[0]):
    date_str = train.at[i,"yyyymmdd"]
    tmp = date_str.split('-')
    date_formatted = datetime.date(int(tmp[0]), int(tmp[1]), int(tmp[2]))
    delta = datetime.date(2020, 1, 1) -date_formatted
    train.at[i,"dateDelta"] = delta.days

In [53]:
train["hour"] = ""
for i in range(train.shape[0]):
    time = train.at[i,"time"]
    tmp = int(time.split(':')[0])
    train.at[i,"hour"] = tmp

In [55]:
train

,id,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A1-level,...,B3-rank,B3-level,B4-weapon,B4-rank,B4-level,y,yyyymmdd,time,dateDelta,hour
0,1,2019-10-15T20:00:00+00:00,5.0.1,regular,standard,nawabari,sumeshi,sshooter_becchu,NaN,139,...,NaN,68.0,sharp_neo,NaN,31.0,1,2019-10-15,20:00:00+00:00,78,20
1,2,2019-12-14T04:00:00+00:00,5.0.1,regular,standard,nawabari,arowana,parashelter_sorella,NaN,198,...,NaN,168.0,sputtery_clear,NaN,151.0,0,2019-12-14,04:00:00+00:00,18,4
2,3,2019-12-25T14:00:00+00:00,5.0.1,gachi,standard,hoko,ama,nzap89,a-,114,...,a-,160.0,dualsweeper_custom,a,126.0,0,2019-12-25,14:00:00+00:00,7,14
3,4,2019-11-11T14:00:00+00:00,5.0.1,regular,standard,nawabari,engawa,bamboo14mk1,NaN,336,...,NaN,194.0,hotblaster_custom,NaN,391.0,0,2019-11-11,14:00:00+00:00,51,14
4,5,2019-12-14T06:00:00+00:00,5.0.1,gachi,standard,hoko,chozame,bold_7,x,299,...,x,246.0,wakaba,x,160.0,1,2019-12-14,06:00:00+00:00,18,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,66121,2019-10-11T10:00:00+00:00,5.0.1,gachi,standard,area,devon,hokusai_becchu,a,66,...,a,38.0,wakaba,a+,283.0,1,2019-10-11,10:00:00+00:00,82,10
66121,66122,2019-12-04T00:00:00+00:00,5.0.1,gachi,standard,asari,otoro,hokusai_becchu,s+,204,...,s+,188.0,nzap89,s+,170.0,1,2019-12-04,00:00:00+00:00,28,0
66122,66123,2019-10-22T00:00:00+00:00,5.0.1,gachi,standard,hoko,bbass,prime_collabo,a+,116,...,a+,191.0,hissen_hue,a+,139.0,1,2019-10-22,00:00:00+00:00,71,0
66123,66124,2019-12-11T00:00:00+00:00,5.0.1,gachi,standard,area,fujitsubo,heroroller_replica,s+,210,...,s+,223.0,sharp_neo,s+,194.0,0,2019-12-11,00:00:00+00:00,21,0


In [60]:
train.to_csv(Output + '/trainMod.csv', index=False)

In [56]:
test[["game-ver"]].apply(pd.value_counts)

,game-ver
5.0.1,28340


In [58]:
test[["period"]].apply(pd.value_counts)

,period
2019-10-12T12:00:00+00:00,102
2019-11-06T14:00:00+00:00,96
2019-12-12T14:00:00+00:00,94
2019-12-14T16:00:00+00:00,92
2019-10-14T14:00:00+00:00,92
...,...
2019-12-13T04:00:00+00:00,1
2019-11-21T22:00:00+00:00,1
2019-11-21T02:00:00+00:00,1
2019-11-16T00:00:00+00:00,1


In [61]:
test['yyyymmdd'] = ""
test["time"] = ""
for i in range(test.shape[0]):
    tmp = test.at[i,"period"].split('T')
    test.at[i,"yyyymmdd"] = tmp[0]
    test.at[i,"time"] = tmp[1]


In [62]:
test[["yyyymmdd"]].apply(pd.value_counts)

,yyyymmdd
2019-10-12,540
2019-12-14,532
2019-10-19,529
2019-12-08,485
2020-01-04,481
...,...
2019-11-21,193
2019-12-11,176
2019-11-20,167
2019-11-18,147


In [63]:
test[["time"]].apply(pd.value_counts)

,time
14:00:00+00:00,5180
12:00:00+00:00,4414
16:00:00+00:00,3683
10:00:00+00:00,2750
08:00:00+00:00,2108
06:00:00+00:00,2026
18:00:00+00:00,1667
04:00:00+00:00,1589
02:00:00+00:00,1353
00:00:00+00:00,1311


In [64]:
test["dateDelta"] = ""
for i in range(test.shape[0]):
    date_str = test.at[i,"yyyymmdd"]
    tmp = date_str.split('-')
    date_formatted = datetime.date(int(tmp[0]), int(tmp[1]), int(tmp[2]))
    delta = datetime.date(2020, 1, 1) -date_formatted
    test.at[i,"dateDelta"] = delta.days

test["hour"] = ""
for i in range(test.shape[0]):
    time = test.at[i,"time"]
    tmp = int(time.split(':')[0])
    test.at[i,"hour"] = tmp

In [65]:
test

,id,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A1-level,...,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level,yyyymmdd,time,dateDelta,hour
0,1,2019-12-17T12:00:00+00:00,5.0.1,gachi,standard,area,hakofugu,prime_collabo,x,174,...,l3reelgun_d,x,213.0,nzap89,x,306.0,2019-12-17,12:00:00+00:00,15,12
1,2,2019-11-25T16:00:00+00:00,5.0.1,gachi,standard,asari,anchovy,prime_becchu,s+,363,...,prime_becchu,s+,147.0,splatroller,s+,156.0,2019-11-25,16:00:00+00:00,37,16
2,3,2019-10-22T08:00:00+00:00,5.0.1,gachi,standard,area,mutsugoro,furo_deco,s,116,...,nzap83,s,183.0,promodeler_pg,s,105.0,2019-10-22,08:00:00+00:00,71,8
3,4,2019-12-30T04:00:00+00:00,5.0.1,gachi,standard,asari,devon,prime_becchu,s+,192,...,soytuber,s+,172.0,prime,s+,70.0,2019-12-30,04:00:00+00:00,2,4
4,5,2019-10-15T16:00:00+00:00,5.0.1,gachi,standard,yagura,anchovy,l3reelgun_d,x,267,...,splatscope,x,50.0,screwslosher_becchu,x,287.0,2019-10-15,16:00:00+00:00,78,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28335,28336,2019-10-11T16:00:00+00:00,5.0.1,gachi,standard,yagura,battera,prime_becchu,x,67,...,prime_becchu,x,240.0,nzap83,x,150.0,2019-10-11,16:00:00+00:00,82,16
28336,28337,2019-10-20T12:00:00+00:00,5.0.1,gachi,standard,yagura,anchovy,longblaster_necro,c-,25,...,nzap89,c-,17.0,dualsweeper,c+,35.0,2019-10-20,12:00:00+00:00,73,12
28337,28338,2019-12-14T00:00:00+00:00,5.0.1,gachi,standard,yagura,zatou,furo,x,386,...,carbon_deco,x,257.0,l3reelgun_d,x,243.0,2019-12-14,00:00:00+00:00,18,0
28338,28339,2019-10-12T04:00:00+00:00,5.0.1,gachi,standard,asari,hokke,maneuver_becchu,s+,185,...,hokusai,s+,152.0,ochiba,s+,130.0,2019-10-12,04:00:00+00:00,81,4


In [66]:
test.to_csv(Output + '/testMod.csv', index=False)

In [3]:
train = pd.read_csv(DataSplatoon + '/trainMod.csv')
test = pd.read_csv(DataSplatoon + '/testMod.csv')

In [6]:
train.columns

Index(['id', 'period', 'game-ver', 'lobby-mode', 'lobby', 'mode', 'stage',
       'A1-weapon', 'A1-rank', 'A1-level', 'A2-weapon', 'A2-rank', 'A2-level',
       'A3-weapon', 'A3-rank', 'A3-level', 'A4-weapon', 'A4-rank', 'A4-level',
       'B1-weapon', 'B1-rank', 'B1-level', 'B2-weapon', 'B2-rank', 'B2-level',
       'B3-weapon', 'B3-rank', 'B3-level', 'B4-weapon', 'B4-rank', 'B4-level',
       'y', 'yyyymmdd', 'time', 'dateDelta', 'hour'],
      dtype='object')

In [4]:
train

,id,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A1-level,...,B3-rank,B3-level,B4-weapon,B4-rank,B4-level,y,yyyymmdd,time,dateDelta,hour
0,1,2019-10-15T20:00:00+00:00,5.0.1,regular,standard,nawabari,sumeshi,sshooter_becchu,NaN,139,...,NaN,68.0,sharp_neo,NaN,31.0,1,2019-10-15,20:00:00+00:00,78,20
1,2,2019-12-14T04:00:00+00:00,5.0.1,regular,standard,nawabari,arowana,parashelter_sorella,NaN,198,...,NaN,168.0,sputtery_clear,NaN,151.0,0,2019-12-14,04:00:00+00:00,18,4
2,3,2019-12-25T14:00:00+00:00,5.0.1,gachi,standard,hoko,ama,nzap89,a-,114,...,a-,160.0,dualsweeper_custom,a,126.0,0,2019-12-25,14:00:00+00:00,7,14
3,4,2019-11-11T14:00:00+00:00,5.0.1,regular,standard,nawabari,engawa,bamboo14mk1,NaN,336,...,NaN,194.0,hotblaster_custom,NaN,391.0,0,2019-11-11,14:00:00+00:00,51,14
4,5,2019-12-14T06:00:00+00:00,5.0.1,gachi,standard,hoko,chozame,bold_7,x,299,...,x,246.0,wakaba,x,160.0,1,2019-12-14,06:00:00+00:00,18,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,66121,2019-10-11T10:00:00+00:00,5.0.1,gachi,standard,area,devon,hokusai_becchu,a,66,...,a,38.0,wakaba,a+,283.0,1,2019-10-11,10:00:00+00:00,82,10
66121,66122,2019-12-04T00:00:00+00:00,5.0.1,gachi,standard,asari,otoro,hokusai_becchu,s+,204,...,s+,188.0,nzap89,s+,170.0,1,2019-12-04,00:00:00+00:00,28,0
66122,66123,2019-10-22T00:00:00+00:00,5.0.1,gachi,standard,hoko,bbass,prime_collabo,a+,116,...,a+,191.0,hissen_hue,a+,139.0,1,2019-10-22,00:00:00+00:00,71,0
66123,66124,2019-12-11T00:00:00+00:00,5.0.1,gachi,standard,area,fujitsubo,heroroller_replica,s+,210,...,s+,223.0,sharp_neo,s+,194.0,0,2019-12-11,00:00:00+00:00,21,0


In [5]:
test

,id,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A1-level,...,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level,yyyymmdd,time,dateDelta,hour
0,1,2019-12-17T12:00:00+00:00,5.0.1,gachi,standard,area,hakofugu,prime_collabo,x,174,...,l3reelgun_d,x,213.0,nzap89,x,306.0,2019-12-17,12:00:00+00:00,15,12
1,2,2019-11-25T16:00:00+00:00,5.0.1,gachi,standard,asari,anchovy,prime_becchu,s+,363,...,prime_becchu,s+,147.0,splatroller,s+,156.0,2019-11-25,16:00:00+00:00,37,16
2,3,2019-10-22T08:00:00+00:00,5.0.1,gachi,standard,area,mutsugoro,furo_deco,s,116,...,nzap83,s,183.0,promodeler_pg,s,105.0,2019-10-22,08:00:00+00:00,71,8
3,4,2019-12-30T04:00:00+00:00,5.0.1,gachi,standard,asari,devon,prime_becchu,s+,192,...,soytuber,s+,172.0,prime,s+,70.0,2019-12-30,04:00:00+00:00,2,4
4,5,2019-10-15T16:00:00+00:00,5.0.1,gachi,standard,yagura,anchovy,l3reelgun_d,x,267,...,splatscope,x,50.0,screwslosher_becchu,x,287.0,2019-10-15,16:00:00+00:00,78,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28335,28336,2019-10-11T16:00:00+00:00,5.0.1,gachi,standard,yagura,battera,prime_becchu,x,67,...,prime_becchu,x,240.0,nzap83,x,150.0,2019-10-11,16:00:00+00:00,82,16
28336,28337,2019-10-20T12:00:00+00:00,5.0.1,gachi,standard,yagura,anchovy,longblaster_necro,c-,25,...,nzap89,c-,17.0,dualsweeper,c+,35.0,2019-10-20,12:00:00+00:00,73,12
28337,28338,2019-12-14T00:00:00+00:00,5.0.1,gachi,standard,yagura,zatou,furo,x,386,...,carbon_deco,x,257.0,l3reelgun_d,x,243.0,2019-12-14,00:00:00+00:00,18,0
28338,28339,2019-10-12T04:00:00+00:00,5.0.1,gachi,standard,asari,hokke,maneuver_becchu,s+,185,...,hokusai,s+,152.0,ochiba,s+,130.0,2019-10-12,04:00:00+00:00,81,4


In [8]:
train[["lobby-mode"]].apply(pd.value_counts)

,lobby-mode
gachi,51681
regular,14444


In [9]:
train[["lobby"]].apply(pd.value_counts)

,lobby
standard,66125


In [10]:
train[["mode"]].apply(pd.value_counts)

,mode
area,14753
nawabari,14444
hoko,13585
yagura,12980
asari,10363


In [11]:
train[["stage"]].apply(pd.value_counts)

,stage
ama,3726
chozame,3487
zatou,3451
mutsugoro,3338
shottsuru,3257
kombu,3243
gangaze,3185
sumeshi,3100
manta,3078
battera,2884


In [12]:
train[["A1-weapon"]].apply(pd.value_counts)

,A1-weapon
prime_becchu,6628
nzap85,4036
dualsweeper_custom,2468
screwslosher_becchu,2129
bold_7,2022
...,...
soytuber_custom,8
heroslosher_replica,5
heroblaster_replica,4
bottlegeyser,3


In [13]:
train[["A1-rank"]].apply(pd.value_counts)

,A1-rank
x,24736
s+,13241
s,5164
a+,3177
a,1540
a-,1368
b+,1215
b,649
b-,415
c+,117


In [14]:
train[["A1-level"]].apply(pd.value_counts)

,A1-level
198,706
297,706
189,499
187,475
188,469
...,...
6,4
7,4
4,3
405,3


In [15]:
train[["y"]].apply(pd.value_counts)

,y
1,34696
0,31429


In [16]:
train[["yyyymmdd"]].apply(pd.value_counts)

,yyyymmdd
2019-10-19,1268
2019-10-12,1232
2019-11-09,1204
2020-01-05,1153
2019-11-03,1131
...,...
2019-11-21,443
2019-11-20,429
2019-11-19,390
2019-11-18,366


In [17]:
train[["time"]].apply(pd.value_counts)

,time
14:00:00+00:00,12016
12:00:00+00:00,10205
16:00:00+00:00,8651
10:00:00+00:00,6565
08:00:00+00:00,4948
06:00:00+00:00,4699
18:00:00+00:00,3933
04:00:00+00:00,3775
02:00:00+00:00,3001
20:00:00+00:00,2993


In [18]:
train[["dateDelta"]].apply(pd.value_counts)

,dateDelta
74,1268
81,1232
53,1204
-4,1153
59,1131
...,...
41,443
42,429
43,390
44,366


In [19]:
train[["hour"]].apply(pd.value_counts)

,hour
14,12016
12,10205
16,8651
10,6565
8,4948
6,4699
18,3933
4,3775
2,3001
20,2993


## 列名をDBに合わせる

In [16]:
train = pd.read_csv(DataSplatoon + '/trainMod.csv')
test = pd.read_csv(DataSplatoon + '/testMod.csv')

In [17]:
train.columns.shape

(36,)

In [18]:
train.columns

Index(['id', 'period', 'game-ver', 'lobby_mode', 'lobby', 'mode', 'stage',
       'A1_weapon', 'A1_rank', 'A1_level', 'A2_weapon', 'A2_rank', 'A2_level',
       'A3_weapon', 'A3_rank', 'A3_level', 'A4_weapon', 'A4_rank', 'A4_level',
       'B1_weapon', 'B1_rank', 'B1_level', 'B2_weapon', 'B2_rank', 'B2_level',
       'B3_weapon', 'B3_rank', 'B3_level', 'B4_weapon', 'B4_rank', 'B4_level',
       'y', 'yyyymmdd', 'time', 'dateDelta', 'hour'],
      dtype='object')

In [19]:
test.columns

Index(['id', 'period', 'game-ver', 'lobby_mode', 'lobby', 'mode', 'stage',
       'A1_weapon', 'A1_rank', 'A1_level', 'A2_weapon', 'A2_rank', 'A2_level',
       'A3_weapon', 'A3_rank', 'A3_level', 'A4_weapon', 'A4_rank', 'A4_level',
       'B1_weapon', 'B1_rank', 'B1_level', 'B2_weapon', 'B2_rank', 'B2_level',
       'B3_weapon', 'B3_rank', 'B3_level', 'B4_weapon', 'B4_rank', 'B4_level',
       'yyyymmdd', 'time', 'dateDelta', 'hour'],
      dtype='object')

In [20]:
train = train.drop(["game-ver", "lobby", "period"], axis=1)
train

,id,lobby_mode,mode,stage,A1_weapon,A1_rank,A1_level,A2_weapon,A2_rank,A2_level,...,B3_rank,B3_level,B4_weapon,B4_rank,B4_level,y,yyyymmdd,time,dateDelta,hour
0,1,regular,nawabari,sumeshi,sshooter_becchu,NaN,139,soytuber_custom,NaN,118.0,...,NaN,68.0,sharp_neo,NaN,31.0,1,2019-10-15,20:00:00+00:00,78,20
1,2,regular,nawabari,arowana,parashelter_sorella,NaN,198,jetsweeper,NaN,77.0,...,NaN,168.0,sputtery_clear,NaN,151.0,0,2019-12-14,04:00:00+00:00,18,4
2,3,gachi,hoko,ama,nzap89,a-,114,quadhopper_black,a,68.0,...,a-,160.0,dualsweeper_custom,a,126.0,0,2019-12-25,14:00:00+00:00,7,14
3,4,regular,nawabari,engawa,bamboo14mk1,NaN,336,splatroller_becchu,NaN,131.0,...,NaN,194.0,hotblaster_custom,NaN,391.0,0,2019-11-11,14:00:00+00:00,51,14
4,5,gachi,hoko,chozame,bold_7,x,299,hissen_hue,x,97.0,...,x,246.0,wakaba,x,160.0,1,2019-12-14,06:00:00+00:00,18,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,66121,gachi,area,devon,hokusai_becchu,a,66,soytuber,a,113.0,...,a,38.0,wakaba,a+,283.0,1,2019-10-11,10:00:00+00:00,82,10
66121,66122,gachi,asari,otoro,hokusai_becchu,s+,204,prime_becchu,s+,96.0,...,s+,188.0,nzap89,s+,170.0,1,2019-12-04,00:00:00+00:00,28,0
66122,66123,gachi,hoko,bbass,prime_collabo,a+,116,promodeler_rg,a,86.0,...,a+,191.0,hissen_hue,a+,139.0,1,2019-10-22,00:00:00+00:00,71,0
66123,66124,gachi,area,fujitsubo,heroroller_replica,s+,210,sharp_neo,s+,96.0,...,s+,223.0,sharp_neo,s+,194.0,0,2019-12-11,00:00:00+00:00,21,0


In [21]:
test = test.drop(["game-ver", "lobby", "period"], axis=1)
test

,id,lobby_mode,mode,stage,A1_weapon,A1_rank,A1_level,A2_weapon,A2_rank,A2_level,...,B3_weapon,B3_rank,B3_level,B4_weapon,B4_rank,B4_level,yyyymmdd,time,dateDelta,hour
0,1,gachi,area,hakofugu,prime_collabo,x,174,herospinner_replica,x,130.0,...,l3reelgun_d,x,213.0,nzap89,x,306.0,2019-12-17,12:00:00+00:00,15,12
1,2,gachi,asari,anchovy,prime_becchu,s+,363,nzap89,s+,59.0,...,prime_becchu,s+,147.0,splatroller,s+,156.0,2019-11-25,16:00:00+00:00,37,16
2,3,gachi,area,mutsugoro,furo_deco,s,116,bold,s,125.0,...,nzap83,s,183.0,promodeler_pg,s,105.0,2019-10-22,08:00:00+00:00,71,8
3,4,gachi,asari,devon,prime_becchu,s+,192,splatspinner_collabo,s+,115.0,...,soytuber,s+,172.0,prime,s+,70.0,2019-12-30,04:00:00+00:00,2,4
4,5,gachi,yagura,anchovy,l3reelgun_d,x,267,rapid_becchu,x,223.0,...,splatscope,x,50.0,screwslosher_becchu,x,287.0,2019-10-15,16:00:00+00:00,78,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28335,28336,gachi,yagura,battera,prime_becchu,x,67,splatroller,x,62.0,...,prime_becchu,x,240.0,nzap83,x,150.0,2019-10-11,16:00:00+00:00,82,16
28336,28337,gachi,yagura,anchovy,longblaster_necro,c-,25,splatroller_collabo,c-,18.0,...,nzap89,c-,17.0,dualsweeper,c+,35.0,2019-10-20,12:00:00+00:00,73,12
28337,28338,gachi,yagura,zatou,furo,x,386,kugelschreiber,x,136.0,...,carbon_deco,x,257.0,l3reelgun_d,x,243.0,2019-12-14,00:00:00+00:00,18,0
28338,28339,gachi,asari,hokke,maneuver_becchu,s+,185,wakaba,s+,103.0,...,hokusai,s+,152.0,ochiba,s+,130.0,2019-10-12,04:00:00+00:00,81,4


In [22]:
train.to_csv(DataSplatoon + '/trainDB.csv', index=False)
test.to_csv(DataSplatoon + '/testDB.csv', index=False)

In [32]:
train.dtypes

id              int64
lobby_mode     object
mode           object
stage          object
A1_weapon      object
A1_rank        object
A1_level        int64
A2_weapon      object
A2_rank        object
A2_level      float64
A3_weapon      object
A3_rank        object
A3_level      float64
A4_weapon      object
A4_rank        object
A4_level      float64
B1_weapon      object
B1_rank        object
B1_level        int64
B2_weapon      object
B2_rank        object
B2_level      float64
B3_weapon      object
B3_rank        object
B3_level      float64
B4_weapon      object
B4_rank        object
B4_level      float64
y               int64
yyyymmdd       object
time           object
dateDelta       int64
hour            int64
dtype: object

In [33]:
train["A2_level"] = train["A2_level"].fillna(-1.0).astype(int)

In [39]:
train["A3_level"] = train["A3_level"].fillna(-1.0).astype(int)
train["A4_level"] = train["A4_level"].fillna(-1.0).astype(int)
train["B2_level"] = train["B2_level"].fillna(-1.0).astype(int)
train["B3_level"] = train["B3_level"].fillna(-1.0).astype(int)
train["B4_level"] = train["B4_level"].fillna(-1.0).astype(int)

In [40]:
trA4 = train["A4_level"]

In [41]:
trA4[trA4.isnull()]

Series([], Name: A4_level, dtype: int64)

In [43]:
train.dtypes

id             int64
lobby_mode    object
mode          object
stage         object
A1_weapon     object
A1_rank       object
A1_level       int64
A2_weapon     object
A2_rank       object
A2_level       int64
A3_weapon     object
A3_rank       object
A3_level       int64
A4_weapon     object
A4_rank       object
A4_level       int64
B1_weapon     object
B1_rank       object
B1_level       int64
B2_weapon     object
B2_rank       object
B2_level       int64
B3_weapon     object
B3_rank       object
B3_level       int64
B4_weapon     object
B4_rank       object
B4_level       int64
y              int64
yyyymmdd      object
time          object
dateDelta      int64
hour           int64
dtype: object

In [44]:
test["A2_level"] = test["A2_level"].fillna(-1.0).astype(int)
test["A3_level"] = test["A3_level"].fillna(-1.0).astype(int)
test["A4_level"] = test["A4_level"].fillna(-1.0).astype(int)
test["B2_level"] = test["B2_level"].fillna(-1.0).astype(int)
test["B3_level"] = test["B3_level"].fillna(-1.0).astype(int)
test["B4_level"] = test["B4_level"].fillna(-1.0).astype(int)

In [45]:
test.dtypes

id             int64
lobby_mode    object
mode          object
stage         object
A1_weapon     object
A1_rank       object
A1_level       int64
A2_weapon     object
A2_rank       object
A2_level       int64
A3_weapon     object
A3_rank       object
A3_level       int64
A4_weapon     object
A4_rank       object
A4_level       int64
B1_weapon     object
B1_rank       object
B1_level       int64
B2_weapon     object
B2_rank       object
B2_level       int64
B3_weapon     object
B3_rank       object
B3_level       int64
B4_weapon     object
B4_rank       object
B4_level       int64
yyyymmdd      object
time          object
dateDelta      int64
hour           int64
dtype: object

In [46]:
train.to_csv(DataSplatoon + '/trainDB.csv', index=False)
test.to_csv(DataSplatoon + '/testDB.csv', index=False)

## SQL操作確認

In [3]:
conn = psycopg2.connect("host=localhost port=5432 dbname=Splatoon user=tidal password=tidalryoku")
cur = conn.cursor()

In [4]:
cur.execute('SELECT * FROM train')
rows = cur.fetchall()

In [5]:
train = pd.DataFrame(rows)
colnames = [col.name for col in cur.description]
train.columns = colnames
train

,id,lobby_mode,mode,stage,a1_weapon,a1_rank,a1_level,a2_weapon,a2_rank,a2_level,...,b3_rank,b3_level,b4_weapon,b4_rank,b4_level,yyyymmdd,time,datedelta,hour,y
0,1,regular,nawabari,sumeshi,sshooter_becchu,None,139,soytuber_custom,None,118,...,None,68,sharp_neo,None,31,2019-10-15,20:00:00,78,20,1
1,2,regular,nawabari,arowana,parashelter_sorella,None,198,jetsweeper,None,77,...,None,168,sputtery_clear,None,151,2019-12-14,04:00:00,18,4,0
2,3,gachi,hoko,ama,nzap89,a-,114,quadhopper_black,a,68,...,a-,160,dualsweeper_custom,a,126,2019-12-25,14:00:00,7,14,0
3,4,regular,nawabari,engawa,bamboo14mk1,None,336,splatroller_becchu,None,131,...,None,194,hotblaster_custom,None,391,2019-11-11,14:00:00,51,14,0
4,5,gachi,hoko,chozame,bold_7,x,299,hissen_hue,x,97,...,x,246,wakaba,x,160,2019-12-14,06:00:00,18,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,66121,gachi,area,devon,hokusai_becchu,a,66,soytuber,a,113,...,a,38,wakaba,a+,283,2019-10-11,10:00:00,82,10,1
66121,66122,gachi,asari,otoro,hokusai_becchu,s+,204,prime_becchu,s+,96,...,s+,188,nzap89,s+,170,2019-12-04,00:00:00,28,0,1
66122,66123,gachi,hoko,bbass,prime_collabo,a+,116,promodeler_rg,a,86,...,a+,191,hissen_hue,a+,139,2019-10-22,00:00:00,71,0,1
66123,66124,gachi,area,fujitsubo,heroroller_replica,s+,210,sharp_neo,s+,96,...,s+,223,sharp_neo,s+,194,2019-12-11,00:00:00,21,0,0


In [6]:
cur.execute('SELECT * FROM test')
rows = cur.fetchall()
test = pd.DataFrame(rows)
colnames = [col.name for col in cur.description]
test.columns = colnames
test

,id,lobby_mode,mode,stage,a1_weapon,a1_rank,a1_level,a2_weapon,a2_rank,a2_level,...,b3_weapon,b3_rank,b3_level,b4_weapon,b4_rank,b4_level,yyyymmdd,time,datedelta,hour
0,1,gachi,area,hakofugu,prime_collabo,x,174,herospinner_replica,x,130,...,l3reelgun_d,x,213,nzap89,x,306,2019-12-17,12:00:00,15,12
1,2,gachi,asari,anchovy,prime_becchu,s+,363,nzap89,s+,59,...,prime_becchu,s+,147,splatroller,s+,156,2019-11-25,16:00:00,37,16
2,3,gachi,area,mutsugoro,furo_deco,s,116,bold,s,125,...,nzap83,s,183,promodeler_pg,s,105,2019-10-22,08:00:00,71,8
3,4,gachi,asari,devon,prime_becchu,s+,192,splatspinner_collabo,s+,115,...,soytuber,s+,172,prime,s+,70,2019-12-30,04:00:00,2,4
4,5,gachi,yagura,anchovy,l3reelgun_d,x,267,rapid_becchu,x,223,...,splatscope,x,50,screwslosher_becchu,x,287,2019-10-15,16:00:00,78,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28335,28336,gachi,yagura,battera,prime_becchu,x,67,splatroller,x,62,...,prime_becchu,x,240,nzap83,x,150,2019-10-11,16:00:00,82,16
28336,28337,gachi,yagura,anchovy,longblaster_necro,c-,25,splatroller_collabo,c-,18,...,nzap89,c-,17,dualsweeper,c+,35,2019-10-20,12:00:00,73,12
28337,28338,gachi,yagura,zatou,furo,x,386,kugelschreiber,x,136,...,carbon_deco,x,257,l3reelgun_d,x,243,2019-12-14,00:00:00,18,0
28338,28339,gachi,asari,hokke,maneuver_becchu,s+,185,wakaba,s+,103,...,hokusai,s+,152,ochiba,s+,130,2019-10-12,04:00:00,81,4


In [7]:
cur.close()
conn.close()


##